In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk  
from itertools import repeat
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.optimizers import Adam
import keras
from tensorflow.keras import backend
from keras import backend as K
from keras import regularizers
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import words
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import sys
import gc
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer


Using TensorFlow backend.


In [2]:
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
# Predador dataset TRAIN
#"""
df  = pd.read_csv('OficialData/train_nonFilter.csv',encoding='utf-8',header=0)
df["0"].fillna("", inplace=True)
print(df["0"][0])
Xss_train = df["0"]
print(Xss_train.shape)
df.head()
#"""


Hola. hi. whats up? not a ton. you? same.  being lazy.  M or f? F. Ditto, I&apos;ve done absolutely nothing with my day besides watching stuff on Hulu. M here.  Just got home from weekend trip.  Tired. Oh, cool. Family thing? yeah. a &amp; l? Gotta love those. 17, Hawaii. and yourself? Uh oh.  older. 30 Been to Hawaii. whoops xD It&apos;s nice, isn&apos;t it? Yeah.  Always enjoy visiting. Which Island you on Oahu? married? i&apos;m assuming since you went on a &apos;family&apos; trip :p yeah. Just found this site a few days ago. Yeah, Oahu. Curious to the whole &quot;random thing&quot; Pretty crazy the individuals you meet, isn&apos;t it? It&apos;s been eye opening for sure. Yeah, I hear you. I&apos;m pretty open to meet/talk to anyone. But pretty clear what most are looking for on this site, it seems. Half the people that strike up a conversation only seem to be interested in more. . risque topics. What more do you expect though, ya know? Yep.  It&apos;s the internet. You seem to take

,Unnamed: 0,0
0,0,Hola. hi. whats up? not a ton. you? same. bei...
1,1,asuu lonte koe hi
2,2,hi asl m or f m
3,3,happy is ayuppie word. a yuppie * yuppie hehe....
4,4,ask me 5 questions and i will answer them trut...


In [4]:
# Predador Labels train
#"""
df = pd.DataFrame()
df  = pd.read_csv('OficialData/train_labels_nonFilter.csv',encoding='utf-8',header=0)
Ys_train = df["0"]
print(Ys_train.shape)
Ys_train = np.asarray(Ys_train)
df.head()
#"""

(66927,)


,0
0,0
1,0
2,0
3,0
4,0


In [5]:
# Predador dataset TEST
#"""
df  = pd.read_csv('OficialData/test_nonFilter.csv',encoding='utf-8',header=0)
df["0"].fillna("", inplace=True)
print(df["0"][0])
Xss_test = df["0"]
print(Xss_test.shape)
df.head()
#"""

bugmail: [Bug 6978] New: Mark eof-terminated script elements as malformed &lt;http://lists.w3.org/Archives/Public/public-html-bugzilla/2009May/0049.html&gt; Henri, can I ask you a Firefox build question (Windows)? 60659cfda992013e610f285c46692d28: sure, but I probably don't know the answer It appears the build runs through, it creates a firefox.exe in dist/bin when I start it, I get my standard install of FF (3.0.10) instead... Same if I make a package, unzip it, and start from there... 60659cfda992013e610f285c46692d28: do you already have the usual Firefox open? Likely So do I need to close all instances? ...other... 60659cfda992013e610f285c46692d28: at least with the Linux version, you need to (unless you run a different profile, which may be advisable anyway) Good point. Will be gone for a moment due to Chatzilla. Henri, that was the problem. ok. good that I can guess some windows things from linux behavior ah yeah, Firefox is annoying with that 60659cfda992013e610f285c46692d28, I w

,Unnamed: 0,0
0,0,bugmail: [Bug 6978] New: Mark eof-terminated s...
1,1,back ok babe what happend babe dog wanted out ...
2,2,hello boys are shit that they are hi sorry i w...
3,3,hi hii hous it going? w r u from ?
4,4,"hello there how are ya? hey so, where are you ..."


In [6]:
# Predador Labels Test
#"""
df = pd.DataFrame()
df  = pd.read_csv('OficialData/test_labels_nonFilter.csv',encoding='utf-8',header=0)
Ys_test = df["0"]
print(Ys_test.shape)
Ys_test = np.asarray(Ys_test)
df.head()

(155128,)


,0
0,0
1,1
2,0
3,0
4,0


In [7]:
posis_train = np.argwhere(Ys_train==1)
negis_train = np.argwhere(Ys_train==0)
print("Predadors Train: ",len(posis_train)," No Predadors Train: ", len(negis_train))
posis_test = np.argwhere(Ys_test==1)
negis_test = np.argwhere(Ys_test==0)
print("Predadors Test: ",len(posis_test)," No Predadors Test: ", len(negis_test))

Predadors Train:  2016  No Predadors Train:  64911
Predadors Test:  3737  No Predadors Test:  151391


Pre processing

In [8]:
# Use them or not ? put more characters
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [dD\)\]\(\]/\\oOpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 

In [9]:
"""
def pre_processing_matlab(conversation):
    # First pre-processing: Matlab
    conversation = conversation.replace('"','') 
    conversation = conversation.replace('.',' ') 
    conversation = conversation.replace("''",' ')
    conversation = conversation.lower()
    conversation = conversation.replace('[^\w\s]',' ') 
    
    # Second pre-processing: TMG

    # Remove punctuation as: !?&
    conversation = conversation.translate(str.maketrans('', '', string.punctuation))
    # Remove numerbs 
    ######conversation = re.sub(r'\b[0-9]+\b', '', conversation)#and alphanumerics too : re.sub('[0-9]+', '', i)
    # Remove  alphanumerics
    conversation = re.sub('[0-9]+', '', conversation)   
    
    # Tokenize and remove SW
    tokens = word_tokenize(conversation)
    conversation = [i for i in tokens if not i in stop_words]
    
    # Applying Potter Steamming
    #conversation = [porter.stem(i) for i in conversation ]
    
    # Applying Lemmatization
    conversation = [wordnet_lemmatizer.lemmatize(i, pos="v") for i in conversation ]
    
    conversation = " ".join(conversation)
    
    return conversation
'heey (: how&apos;re you? well hi alright asl? '
"""



def pre_processing(conversation):
    # Remove  HTML tags
    conversation = re.sub('<[^>]+>+', '', conversation) 
    
    # Remove  @-mentions
    conversation = re.sub('(?:@[\w_]+)', '', conversation) 
    
    # Remove  hash-tags
    conversation = re.sub("(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", '', conversation) 
    
    # Remove  URL
    conversation = re.sub('http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+''', '', conversation)

    #"""
    # Remove numerbs 
    conversation = re.sub(r'\b[0-9]+\b', '', conversation)#and alphanumerics too : re.sub('[0-9]+', '', i)
    #"""
    # Remove  number of strings
    conversation = re.sub('[0-9]+', '', conversation)   
    
    # Remove strings with numbers (all), this may be important whent we have dicc abbrev e.g sk8
    #conversation = re.sub('\w*\d\w*', '', conversation) 

    # Tokenize and remove emoticons
    tokens = tokenize(conversation) # Special tokenization
    tokens = [emoticon_re.sub(r'', i) for i in tokens ]
    conversation = " ".join(tokens)
    
    # Tokenize and remove SW
    conversation = conversation.lower()
    tokens = word_tokenize(conversation)
    conversation = [i for i in tokens if not i in stop_words]
    #conversation = [i for i in tokens if i not in all_sw]
    
    # Remove words with len < 3 and > 20
    #conversation = [i for i in conversation if len(i) > 3 and len(i) <= 20]
    #conversation = [i for i in conversation if len(i) > 3]
    
    
    
    # Applying Potter Steamming
    #conversation = [porter.stem(i) for i in conversation ]
    
    # Applying Lemmatization
    #conversation = [wordnet_lemmatizer.lemmatize(i, pos="v") for i in conversation ]
    
    conversation = " ".join(conversation)
    
    #print(conversation)
    
    # Remove punctuation as: !?&
    conversation = conversation.translate(str.maketrans('', '', string.punctuation))
   
    
    return conversation


In [10]:
t = 'heey (: how&apos;re you? well hi alright asl? '
print(pre_processing(t))

heey    apos   well hi alright asl 


In [11]:
def pre_processing_dani(documents,labels):
    docs = []
    Y = []
    empty_docs = 0
    for id,conversation in enumerate(documents):
        clean_conversation = pre_processing(conversation)
        if len (clean_conversation) == 0:
            empty_docs += 1
        else:
            docs.append(clean_conversation)
            Y.append(labels[id])
    print("Empty Documetns: ",empty_docs)
    return docs,Y

In [12]:
print("Before: ",Xss_train[0])
start = time.time()
doc_train,new_Y_train = pre_processing_dani(Xss_train,Ys_train)
print("Train Documents, There are :",len(doc_train),": ", doc_train[0]," ",len(new_Y_train))
end = time.time()
print("Time is: ",end - start)

Before:  Hola. hi. whats up? not a ton. you? same.  being lazy.  M or f? F. Ditto, I&apos;ve done absolutely nothing with my day besides watching stuff on Hulu. M here.  Just got home from weekend trip.  Tired. Oh, cool. Family thing? yeah. a &amp; l? Gotta love those. 17, Hawaii. and yourself? Uh oh.  older. 30 Been to Hawaii. whoops xD It&apos;s nice, isn&apos;t it? Yeah.  Always enjoy visiting. Which Island you on Oahu? married? i&apos;m assuming since you went on a &apos;family&apos; trip :p yeah. Just found this site a few days ago. Yeah, Oahu. Curious to the whole &quot;random thing&quot; Pretty crazy the individuals you meet, isn&apos;t it? It&apos;s been eye opening for sure. Yeah, I hear you. I&apos;m pretty open to meet/talk to anyone. But pretty clear what most are looking for on this site, it seems. Half the people that strike up a conversation only seem to be interested in more. . risque topics. What more do you expect though, ya know? Yep.  It&apos;s the internet. You see

In [13]:
print("Before: ",Xss_test[0])
start = time.time()
doc_test,new_Y_test = pre_processing_dani(Xss_test,Ys_test)
print("Test Documents, There are :",len(doc_test),": ", doc_test[0]," ",len(new_Y_test))
end = time.time()
print("Time is: ",end - start)

Before:  bugmail: [Bug 6978] New: Mark eof-terminated script elements as malformed &lt;http://lists.w3.org/Archives/Public/public-html-bugzilla/2009May/0049.html&gt; Henri, can I ask you a Firefox build question (Windows)? 60659cfda992013e610f285c46692d28: sure, but I probably don't know the answer It appears the build runs through, it creates a firefox.exe in dist/bin when I start it, I get my standard install of FF (3.0.10) instead... Same if I make a package, unzip it, and start from there... 60659cfda992013e610f285c46692d28: do you already have the usual Firefox open? Likely So do I need to close all instances? ...other... 60659cfda992013e610f285c46692d28: at least with the Linux version, you need to (unless you run a different profile, which may be advisable anyway) Good point. Will be gone for a moment due to Chatzilla. Henri, that was the problem. ok. good that I can guess some windows things from linux behavior ah yeah, Firefox is annoying with that 60659cfda992013e610f285c4669

In [14]:
#"""
Y_train = np.asarray(new_Y_train)
Y_test = np.asarray(new_Y_test)

posi_train = np.argwhere(Y_train==1)
posi_test  = np.argwhere(Y_test==1)
nega_train = np.argwhere(Y_train==0)
nega_test  = np.argwhere(Y_test==0)
print("Predadors: ",len(posi_train)," No Predadors: ", len(nega_train))
print("Predadors: ",len(posi_test)," No Predadors: ", len(nega_test))
#"""print("Doc Train:",len(doc_train)," Labels Train: ",len(new_Y_train))
print("Doc Test :",len(doc_test)," Labels Test : ",len(new_Y_test))



Predadors:  1974  No Predadors:  64581
Predadors:  3685  No Predadors:  150702
Doc Test : 154387  Labels Test :  154387


In [15]:
word_gram = 1
vectorizer = CountVectorizer(dtype=np.int64,min_df=5,strip_accents="ascii") 
X = vectorizer.fit_transform(doc_train)
X_train = X.toarray()
print(X_train.shape)

(66555, 21238)


In [16]:
DicC = vectorizer.get_feature_names()
print(len(DicC)," ",DicC[765])

21238   angeles


In [17]:
vectorizer = CountVectorizer(vocabulary=DicC,dtype=np.int64,strip_accents="ascii") 
X = vectorizer.fit_transform(doc_test)
print(X.shape)
X_test = X.toarray()

(154387, 21238)


TF IDF SUBLINEAR

In [18]:
predador_convs , normal_convs = [],[]
idx = 0
for conversation in doc_train:
    if Y_train[idx] == 1:
        predador_convs.append(conversation)
    else:
        normal_convs.append(conversation)
    idx += 1
print("Predadors: ",len(predador_convs)," No Predadors: ", len(normal_convs))


Predadors:  1974  No Predadors:  64581


In [19]:
corpus = predador_convs
vectorizer = TfidfVectorizer(vocabulary = DicC, sublinear_tf=True,use_idf=True)
X_predador = vectorizer.fit_transform(corpus)
X_train_P = X_predador.toarray()
print(X_train_P.shape)

(1974, 21238)


In [20]:
corpus = normal_convs
vectorizer = vectorizer = TfidfVectorizer(vocabulary = DicC, sublinear_tf=True,use_idf=True)

X_no_predador = vectorizer.fit_transform(corpus)
X_train_NP = X_no_predador.toarray()
print(X_train_NP.shape)


(64581, 21238)


Shuffle Train Data: Optional?

In [21]:
from sklearn.utils import shuffle

#X_train, Y_train = shuffle(X_train, Y_train)
print(X_train.shape)
print(Y_train.shape)

(66555, 21238)
(66555,)


In [22]:
def sklearn_mesures(Y_pred,Y_test,mesure):
    a  = accuracy_score(Y_test, Y_pred, normalize=True)
    p  = precision_score(Y_test, Y_pred , average=mesure)
    r  = recall_score(Y_test, Y_pred , average=mesure)
    f1_c1 = f1_score(Y_test, Y_pred , average=mesure,pos_label=0)
    f1_c2 = f1_score(Y_test, Y_pred , average=mesure,pos_label=1)
    return [f1_c1,f1_c2]#[a,p,r,f1_c1,f1_c2]



In [23]:
def f1_matlab(predict,true,c):
    # This function outputs the F1 per class , this from Matlab
    erx = np.sign(predict) - np.sign(true)
    ofc = np.argwhere(np.sign(true)==c) 
    pfc = np.argwhere(np.sign(predict)==c)

    p = (len(np.argwhere(erx[pfc]==0))/len(pfc))
    r = (len(np.argwhere(erx[ofc]==0))/len(ofc))
    
    tmp = (r + p)

    f = ((2 * p * r) / tmp) * 100
    return [f]

In [24]:
# Making the predicction, and setting threshold to 0.5
def predicction(Y_pred, Y_test):
    Y_pred_bool = []
    Y_pred = np.where(Y_pred > 0.5, 1, 0) 
    for i in Y_pred:
        Y_pred_bool.append(i[0]) 
    Y_pred_bool = np.asarray(Y_pred_bool) 

    non_predadors  = 0
    predadors      = 0
    for v in Y_pred_bool:
        if v == 0:
            non_predadors += 1
        else:
            predadors += 1
        
    if (non_predadors == 0 or predadors == 0):
        print("Empty Label!!!")
        print("Predicctions with sklearn")
        print(sklearn_mesures(Y_pred,Y_test,"binary"))
        return [1,0]
    else:
        
        # Predictions con sklearn mesures and matlab function
        c1_f1 = f1_matlab(Y_pred_bool,Y_test,0)
        c2_f1 = f1_matlab(Y_pred_bool,Y_test,1)
        print("Predicctions with sklearn")
        print(sklearn_mesures(Y_pred,Y_test,"binary"))
        print("Predicctions with Matlab")
        print(c1_f1," ",c2_f1)
        print("Confusion Matrix")
        print(confusion_matrix(Y_pred_bool, Y_test))
    return [c1_f1[0],c2_f1[0]]

In [25]:
# TD-IDF Sublinear
def csa (X,XI,XII,Y):

    num_terms = XI.shape[1]# number of termos, i both X are the same
    num_class = 2
    num_docs = X.shape[0]
    DL = np.sum(X, axis=1) # sum each elemet row, wich is each df element of a document
    DL[DL <1 ] = 1
    TR = np.zeros((num_terms, num_class))
    
   
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        #docs = np.where(Y == j)[0] 
        if j == 0:
            X_aux = XI # no predador
        else :
            X_aux = XII # predador
        for i in range(0,num_terms):
            tf_v = X_aux[:,i] #X[docs,i] # tf_v is tern frecuency of each term
            #print((tf_v.shape)," ",num_terms)
            #-------
            #TF : Dani
            TR[i][j] = sum(tf_v)  
            #-------
            #print ("TR[i][j]: ",TR[i][j])
            if np.isnan(TR[i][j]):
                TR[i][j] = 0
        print("------------------NEW TERM------------------------")
    #TR = np.nan_to_num(TR)
    print("PRB: (",TR.shape[0],",",TR.shape[1],")")

    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    n1[n1 < 1] = 1
    Tik = TR / n1
    #Tik = np.nan_to_num(Tik)

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        suma = sum(Tik[i])
        if suma == 0:
            suma = 1
        Tik[i] = Tik[i] / suma # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X[i] != 0)[0] # extrac idxs =! 0
        #-------
        #TF : AUTOR
        a = X[i][termos] / DL[i]# acces idxs content / len  
        a = np.expand_dims(a, axis=1)
        #-----
        b = (Tik[termos])
        DR[i] = sum(a*b) # alpha (a)* tik; alpha is the df of each term
        print("DR[i]: ",DR[i]," sum: ",DR[i][0]+DR[i][1])
    # Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"    return Tik
    
    

    return DR,Tik
print("Not for now, tf idf")

Not for now, tf idf


In [26]:
start = time.time()
DR,Tik = csa (X_train,X_train_NP,X_train_P,Y_train)
end = time.time()
print("DR: (",DR.shape[0],",",DR.shape[1],")")
print("Time: ",end-start)

------------------NEW TERM------------------------
------------------NEW TERM------------------------
PRB: ( 21238 , 2 )
Tik: ( 21238 , 2 )
DR[i]:  [0.54680873 0.45319127]  sum:  1.0000000000000002
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.86234154 0.13765846]  sum:  1.0
DR[i]:  [0.60562141 0.39437859]  sum:  1.0
DR[i]:  [0.58578146 0.41421854]  sum:  1.0
DR[i]:  [0.81936929 0.18063071]  sum:  0.9999999999999999
DR[i]:  [0.91650986 0.08349014]  sum:  1.0
DR[i]:  [0.60025099 0.39974901]  sum:  1.0
DR[i]:  [0.93117077 0.06882923]  sum:  0.9999999999999999
DR[i]:  [0.96886412 0.03113588]  sum:  1.0
DR[i]:  [0.70172449 0.29827551]  sum:  1.0000000000000002
DR[i]:  [0.68594875 0.31405125]  sum:  0.9999999999999998
DR[i]:  [0.50140228 0.49859772]  sum:  0.9999999999999998
DR[i]:  [0.98842748 0.01157252]  sum:  0.9999999999999999
DR[i]:  [0.68231595 0.31768405]  sum:  1.0
DR[i]:  [0.25570548 0.74429452]  sum:  1.0
DR[i]:  [0.85175284 0.14824716]  sum:  0.9999999999999999
DR[i]:  [

DR[i]:  [0.72461098 0.27538902]  sum:  1.0
DR[i]:  [0.94789412 0.05210588]  sum:  1.0
DR[i]:  [0.70226388 0.29773612]  sum:  0.9999999999999999
DR[i]:  [0.85001243 0.14998757]  sum:  0.9999999999999999
DR[i]:  [0.61893859 0.38106141]  sum:  1.0
DR[i]:  [0.86671239 0.13328761]  sum:  0.9999999999999999
DR[i]:  [0.94256872 0.05743128]  sum:  1.0000000000000002
DR[i]:  [0.46449587 0.53550413]  sum:  1.0
DR[i]:  [0.84112297 0.15887703]  sum:  1.0
DR[i]:  [0.81599373 0.18400627]  sum:  1.0000000000000009
DR[i]:  [0.80102485 0.19897515]  sum:  1.0000000000000004
DR[i]:  [0.26099442 0.73900558]  sum:  0.9999999999999998
DR[i]:  [0.30475171 0.69524829]  sum:  1.0
DR[i]:  [0.72809966 0.27190034]  sum:  1.0
DR[i]:  [0.58144696 0.41855304]  sum:  1.0
DR[i]:  [0.89071885 0.10928115]  sum:  1.0000000000000002
DR[i]:  [0.24470956 0.75529044]  sum:  1.0
DR[i]:  [0.70427372 0.29572628]  sum:  1.0
DR[i]:  [0.51338938 0.48661062]  sum:  0.9999999999999999
DR[i]:  [0.80616098 0.19383902]  sum:  1.0
DR[i]

DR[i]:  [0.84282719 0.15717281]  sum:  1.0000000000000002
DR[i]:  [0.96490331 0.03509669]  sum:  1.0
DR[i]:  [0.94813044 0.05186956]  sum:  1.0
DR[i]:  [0.71630907 0.28369093]  sum:  1.0
DR[i]:  [0.54116861 0.45883139]  sum:  1.0
DR[i]:  [0.85001243 0.14998757]  sum:  0.9999999999999999
DR[i]:  [0.70783536 0.29216464]  sum:  0.9999999999999999
DR[i]:  [0.88574092 0.11425908]  sum:  1.0
DR[i]:  [0.77101154 0.22898846]  sum:  1.0
DR[i]:  [0.55240827 0.44759173]  sum:  1.0
DR[i]:  [0.6717348 0.3282652]  sum:  1.0
DR[i]:  [0.89162157 0.10837843]  sum:  1.0
DR[i]:  [0.64911528 0.35088472]  sum:  1.0
DR[i]:  [0.87317983 0.12682017]  sum:  1.0
DR[i]:  [0.84209915 0.15790085]  sum:  0.9999999999999999
DR[i]:  [0.63671636 0.36328364]  sum:  0.9999999999999999
DR[i]:  [0.85173225 0.14826775]  sum:  0.9999999999999999
DR[i]:  [0.98349233 0.01650767]  sum:  1.0
DR[i]:  [0.90822769 0.09177231]  sum:  0.9999999999999999
DR[i]:  [0.64897504 0.35102496]  sum:  1.0000000000000004
DR[i]:  [0.58781177 0.

DR[i]:  [0.82228319 0.17771681]  sum:  0.9999999999999999
DR[i]:  [0.48200211 0.51799789]  sum:  1.0
DR[i]:  [1. 0.]  sum:  1.0
DR[i]:  [0.64059936 0.35940064]  sum:  1.0
DR[i]:  [0.76405587 0.23594413]  sum:  0.9999999999999999
DR[i]:  [0.81155237 0.18844763]  sum:  1.0
DR[i]:  [1. 0.]  sum:  1.0
DR[i]:  [0.55693878 0.44306122]  sum:  1.0
DR[i]:  [0.70016589 0.29983411]  sum:  1.0000000000000002
DR[i]:  [0.61649626 0.38350374]  sum:  1.0
DR[i]:  [0.71124821 0.28875179]  sum:  0.9999999999999999
DR[i]:  [0.81522852 0.18477148]  sum:  1.0
DR[i]:  [0.66901994 0.33098006]  sum:  1.0
DR[i]:  [0.84791214 0.15208786]  sum:  0.9999999999999999
DR[i]:  [0.90363902 0.09636098]  sum:  1.0000000000000004
DR[i]:  [0.62327056 0.37672944]  sum:  1.0000000000000004
DR[i]:  [0.94445525 0.05554475]  sum:  0.9999999999999999
DR[i]:  [0.78806731 0.21193269]  sum:  1.0
DR[i]:  [0.91134236 0.08865764]  sum:  1.0
DR[i]:  [0.73924019 0.26075981]  sum:  0.9999999999999999
DR[i]:  [0.39716975 0.60283025]  sum:

DR[i]:  [0.81062403 0.18937597]  sum:  0.9999999999999994
DR[i]:  [0.79112848 0.20887152]  sum:  1.0
DR[i]:  [0.52887619 0.47112381]  sum:  1.0
DR[i]:  [0.85719336 0.14280664]  sum:  1.0
DR[i]:  [0.73023947 0.26976053]  sum:  1.0
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.4684112 0.5315888]  sum:  1.0
DR[i]:  [0.76993062 0.23006938]  sum:  1.0
DR[i]:  [0.77777145 0.22222855]  sum:  0.9999999999999998
DR[i]:  [0.86671239 0.13328761]  sum:  0.9999999999999999
DR[i]:  [0.69961128 0.30038872]  sum:  0.9999999999999999
DR[i]:  [0.70783536 0.29216464]  sum:  0.9999999999999999
DR[i]:  [0.15521495 0.84478505]  sum:  1.0
DR[i]:  [0.57860022 0.42139978]  sum:  1.0
DR[i]:  [0.76871001 0.23128999]  sum:  0.9999999999999999
DR[i]:  [0.5854791 0.4145209]  sum:  1.0
DR[i]:  [0.53114142 0.46885858]  sum:  1.0
DR[i]:  [0.71718409 0.28281591]  sum:  1.0
DR[i]:  [0.68515912 0.31484088]  sum:  0.9999999999999998
DR[i]:  [0.69562202 0.30437798]  sum:  0.9999999999999999
DR[i]:  [0.73342478 0.26

...

DR[i]:  [0.46198639 0.53801361]  sum:  0.9999999999999998
DR[i]:  [0.65060383 0.34939617]  sum:  1.0
DR[i]:  [0.80230967 0.19769033]  sum:  0.9999999999999999
DR[i]:  [0.84396944 0.15603056]  sum:  0.9999999999999999
DR[i]:  [0.71336392 0.28663608]  sum:  1.0
DR[i]:  [0.70716729 0.29283271]  sum:  1.0
DR[i]:  [0.79601488 0.20398512]  sum:  0.9999999999999998
DR[i]:  [0.91461145 0.08538855]  sum:  1.0
DR[i]:  [0.27674005 0.72325995]  sum:  1.0
DR[i]:  [0.88394993 0.11605007]  sum:  1.0
DR[i]:  [0.71603152 0.28396848]  sum:  1.0000000000000009
DR[i]:  [0.73529179 0.26470821]  sum:  1.0
DR[i]:  [0.73029583 0.26970417]  sum:  1.0000000000000002
DR[i]:  [0.55039529 0.44960471]  sum:  1.0
DR[i]:  [0.75423058 0.24576942]  sum:  0.9999999999999997
DR[i]:  [0.66352724 0.33647276]  sum:  1.0
DR[i]:  [0.74470822 0.25529178]  sum:  0.9999999999999997
DR[i]:  [0.82209421 0.17790579]  sum:  1.0
DR[i]:  [0.80230967 0.19769033]  sum:  0.9999999999999999
DR[i]:  [0.53285439 0.46714561]  sum:  1.0
DR[i]

DR[i]:  [0.51131876 0.48868124]  sum:  1.0000000000000004
DR[i]:  [0.65236087 0.34763913]  sum:  1.0
DR[i]:  [0.90822769 0.09177231]  sum:  0.9999999999999999
DR[i]:  [0.72792168 0.27207832]  sum:  1.0
DR[i]:  [0.19837567 0.80162433]  sum:  1.0
DR[i]:  [0.7658511 0.2341489]  sum:  1.0
DR[i]:  [0.75416702 0.24583298]  sum:  1.0
DR[i]:  [0.73636399 0.26363601]  sum:  1.0
DR[i]:  [0.67387569 0.32612431]  sum:  1.0
DR[i]:  [0.63184644 0.36815356]  sum:  1.0
DR[i]:  [0.74326223 0.25673777]  sum:  1.0
DR[i]:  [0.76175173 0.23824827]  sum:  1.0
DR[i]:  [0.75696063 0.24303937]  sum:  0.9999999999999996
DR[i]:  [0.50922717 0.49077283]  sum:  1.0
DR[i]:  [0.61124763 0.38875237]  sum:  1.0
DR[i]:  [0.93117077 0.06882923]  sum:  1.0
DR[i]:  [0.66653968 0.33346032]  sum:  1.0
DR[i]:  [0.47126864 0.52873136]  sum:  1.0
DR[i]:  [0.4324801 0.5675199]  sum:  0.9999999999999999
DR[i]:  [0.81488715 0.18511285]  sum:  1.0
DR[i]:  [0.34999951 0.65000049]  sum:  1.0
DR[i]:  [0.72735052 0.27264948]  sum:  1.

DR[i]:  [0.78293332 0.21706668]  sum:  1.0
DR[i]:  [0.81936929 0.18063071]  sum:  0.9999999999999999
DR[i]:  [0.9912583 0.0087417]  sum:  1.0
DR[i]:  [0.89903493 0.10096507]  sum:  0.9999999999999996
DR[i]:  [0.84954683 0.15045317]  sum:  0.9999999999999999
DR[i]:  [0.83675212 0.16324788]  sum:  0.9999999999999999
DR[i]:  [0.79379321 0.20620679]  sum:  0.9999999999999999
DR[i]:  [0.85736593 0.14263407]  sum:  1.0
DR[i]:  [0.74129037 0.25870963]  sum:  1.0
DR[i]:  [0.85483983 0.14516017]  sum:  1.0
DR[i]:  [0.67368865 0.32631135]  sum:  1.0
DR[i]:  [0.88369217 0.11630783]  sum:  1.0
DR[i]:  [1. 0.]  sum:  1.0
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.52719177 0.47280823]  sum:  1.0
DR[i]:  [0.97314392 0.02685608]  sum:  1.0
DR[i]:  [0.82434708 0.17565292]  sum:  0.9999999999999999
DR[i]:  [0.79656777 0.20343223]  sum:  1.0
DR[i]:  [0.66439368 0.33560632]  sum:  1.0
DR[i]:  [0.65458423 0.34541577]  sum:  0.9999999999999991
DR[i]:  [0.47770131 0.52229869]  sum:  1.0
DR[i]:  [0

DR[i]:  [0.53809803 0.46190197]  sum:  0.9999999999999999
DR[i]:  [0.67138571 0.32861429]  sum:  0.9999999999999999
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.89116808 0.10883192]  sum:  0.9999999999999999
DR[i]:  [0.80561987 0.19438013]  sum:  0.9999999999999999
DR[i]:  [0.74548808 0.25451192]  sum:  1.0
DR[i]:  [0.45265057 0.54734943]  sum:  0.9999999999999998
DR[i]:  [0.72277114 0.27722886]  sum:  1.0
DR[i]:  [0.84690716 0.15309284]  sum:  0.9999999999999999
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.84954683 0.15045317]  sum:  0.9999999999999999
DR[i]:  [0.59255545 0.40744455]  sum:  1.0
DR[i]:  [0.84644814 0.15355186]  sum:  1.0
DR[i]:  [0.50869817 0.49130183]  sum:  1.0
DR[i]:  [0.28237466 0.71762534]  sum:  1.0
DR[i]:  [0.95758607 0.04241393]  sum:  1.0
DR[i]:  [0.72482324 0.27517676]  sum:  0.9999999999999996
DR[i]:  [0.80653269 0.19346731]  sum:  1.0
DR[i]:  [0.75094346 0.24905654]  sum:  1.0
DR[i]:  [0.95411385 0.04588615]  sum:  0.9999999999999998
DR[i]

DR[i]:  [0.74031643 0.25968357]  sum:  1.0
DR[i]:  [0.99654511 0.00345489]  sum:  1.0
DR[i]:  [0.91048771 0.08951229]  sum:  1.0
DR[i]:  [0.85194076 0.14805924]  sum:  0.9999999999999999
DR[i]:  [0.81936929 0.18063071]  sum:  0.9999999999999999
DR[i]:  [0.78525005 0.21474995]  sum:  0.9999999999999999
DR[i]:  [0.4872597 0.5127403]  sum:  1.0
DR[i]:  [0.79873369 0.20126631]  sum:  0.9999999999999998
DR[i]:  [0.86234154 0.13765846]  sum:  1.0
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.75725342 0.24274658]  sum:  0.9999999999999999
DR[i]:  [0.81936929 0.18063071]  sum:  0.9999999999999999
DR[i]:  [0.59311856 0.40688144]  sum:  1.0
DR[i]:  [0.91819964 0.08180036]  sum:  1.0
DR[i]:  [0.73342478 0.26657522]  sum:  1.0
DR[i]:  [0.66570953 0.33429047]  sum:  1.0
DR[i]:  [0.52665511 0.47334489]  sum:  0.9999999999999999
DR[i]:  [0.46821547 0.53178453]  sum:  0.9999999999999996
DR[i]:  [0.64044973 0.35955027]  sum:  1.0
DR[i]:  [0.46319629 0.53680371]  sum:  1.0
DR[i]:  [0.90376612 0.

In [27]:
def csa_test (XT,Tik): 
    # Document Test Representation
    num_class = 2
    DLT = np.sum(XT, axis=1)
    DLT[DLT <1 ] = 1
    num_docs = XT.shape[0]
    DRT = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(XT[i] != 0)[0] # extrac idxs =! 0
        a = XT[i][termos] / DLT[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DRT[i] = sum(np.multiply(a,Tik[termos]))
    return DRT

In [28]:
start = time.time()
DRT = csa_test (X_test,Tik)
end = time.time()
print("DRT: (",DRT.shape[0],",",DRT.shape[1],")")
print("Time: ",end-start)

DRT: ( 154387 , 2 )
Time:  17.69056987762451


In [29]:

#del model
#gc.collect()
#K.clear_session()

In [30]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint


In [31]:
#Create NN model
num_class = 2
model = Sequential()
model.add(LSTM(32, input_shape=(num_class, 1)))
#model.add(Bidirectional(LSTM(64, input_shape=(num_class, 1))))
model.add(Dense(1, activation='sigmoid'))
#print(model.summary())

In [32]:
#opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False,decay=1e-14)#adam(decay=1e-14)
opt = keras.optimizers.Adam(decay=1e-14)#adam(decay=1e-14)
#opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])


# checkpoint
filepath="PredadorAllData/Weights/weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [33]:
A = np.reshape(DR, (DR.shape[0],  DR.shape[1],1))
B = np.reshape(DRT, (DRT.shape[0],DRT.shape[1],1))
model.fit(A, Y_train, epochs=30, batch_size=10, callbacks=callbacks_list, validation_data=(B, Y_test),verbose=1)




Train on 66555 samples, validate on 154387 samples
Epoch 1/30





66555/66555 [==============================] - 37s 560us/step - loss: 0.0665 - acc: 0.9766 - val_loss: 0.0537 - val_acc: 0.9808

Epoch 00001: val_acc improved from -inf to 0.98077, saving model to PredadorAllData/Weights/weights.hdf5
Epoch 2/30
66555/66555 [==============================] - 35s 530us/step - loss: 0.0484 - acc: 0.9819 - val_loss: 0.0505 - val_acc: 0.9809

Epoch 00002: val_acc improved from 0.98077 to 0.98085, saving model to PredadorAllData/Weights/weights.hdf5
Epoch 3/30
66555/66555 [==============================] - 36s 535us/step - loss: 0.0468 - acc: 0.9821 - val_loss: 0.0505 - val_acc: 0.9814

Epoch 00003: val_acc improved from 0.98085 to 0.98140, saving model to PredadorAllData/Weights/weights.hdf5
Epoch 4/30
66555/66555 [==============================] - 36s 540us/step - loss: 0.0469 - acc: 0.9821 - val_loss: 0.0499 - val_acc: 0.9814

Epoch 00004: val_acc improved from 0.98140 to 0.98145, savin

In [34]:
model.evaluate(B, Y_test)
Y_pred = model.predict(B, batch_size=10, verbose=0)
predicction(Y_pred, Y_test)

154387/154387 [==============================] - 5s 30us/step
Predicctions with sklearn
[0.9907082551283666, 0.6116962305986696]
Predicctions with Matlab
[99.07082551283666]   [61.169623059866964]
Confusion Matrix
[[149378   1478]
 [  1324   2207]]


[99.07082551283666, 61.169623059866964]

In [35]:
#Create NN model
num_class = 2
model_saved = Sequential()
model_saved.add(LSTM(32, input_shape=(num_class, 1)))
#model.add(Bidirectional(LSTM(64, input_shape=(num_class, 1))))
model_saved.add(Dense(1, activation='sigmoid'))
#print(model.summary())
opt = keras.optimizers.Adam(decay=1e-14)#adam(decay=1e-14)

# load weights
model_saved.load_weights(filepath)
# Compile model (required to make predictions)
model_saved.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print("Created model and loaded weights from file")
model_saved.evaluate(B, Y_test)
Y_pred = model_saved.predict(B, batch_size=10, verbose=0)
predicction(Y_pred, Y_test)

Created model and loaded weights from file
154387/154387 [==============================] - 5s 32us/step
Predicctions with sklearn
[0.9907082551283666, 0.6116962305986696]
Predicctions with Matlab
[99.07082551283666]   [61.169623059866964]
Confusion Matrix
[[149378   1478]
 [  1324   2207]]


[99.07082551283666, 61.169623059866964]

In [36]:
Anotar si dio mas con Sublinear o no, luego
probar con False
y luefo con use idf False .... en oetoria es lo de los autores :)

SyntaxError: invalid syntax (<ipython-input-36-4e97d958c700>, line 1)

In [ ]:
import gc
#del X_train,X_test
gc.collect()

In [ ]:
TF sublinear no IDF == autor
154823/154823 [==============================] - 4s 25us/step
Predicctions with sklearn
[0.9906447316985907, 0.6372045220966083]
Predicctions with Matlab
[99.06447316985907]   [63.72045220966083]
Confusion Matrix
[[149519   1186]
 [  1638   2480]]

[99.06447316985907, 63.72045220966083]
------
TF sublinear si IDF nosso == autor

154823/154823 [==============================] - 4s 25us/step
Predicctions with sklearn
[0.9907418142069262, 0.6394013675654755]
Predicctions with Matlab
[99.07418142069263]   [63.94013675654755]
Confusion Matrix
[[149550   1188]
 [  1607   2478]]

[99.07418142069263, 63.94013675654755]

TF NO sublinear si IDF 
154823/154823 [==============================] - 7s 43us/step
Predicctions with sklearn
[0.9908883223520841, 0.6386554621848739]
Predicctions with Matlab
[99.08883223520841]   [63.86554621848739]
Confusion Matrix
[[149639   1234]
 [  1518   2432]]

[99.08883223520841, 63.86554621848739]

-----------------------------------------------------------

New preprocessing Sublinear no, IDF SI
154387/154387 [==============================] - 4s 24us/step
Predicctions with sklearn
[0.990697149383462, 0.6301478352692714]
Predicctions with Matlab
[99.0697149383462]   [63.01478352692714]
Confusion Matrix
[[149198   1298]
 [  1504   2387]]
[99.0697149383462, 63.01478352692714]

New preprocessing Sublinear si, IDF SI

54387/154387 [==============================] - 5s 30us/step
Predicctions with sklearn
[0.9907082551283666, 0.6116962305986696]
Predicctions with Matlab
[99.07082551283666]   [61.169623059866964]
Confusion Matrix
[[149378   1478]
 [  1324   2207]]
[99.07082551283666, 61.169623059866964]



New preprocessing Sublinear si, IDF no == autor